In [ ]:
# default_exp exec.plotting

In [ ]:
# export
from montlake.atomgeom.features import get_features,get_D_feats_feats
from montlake.atomgeom.utils import get_atoms_4
from montlake.simulations.rigidethanol import get_rigid_ethanol_data
from montlake.utils.utils import get_234_indices, get_atoms3_full, get_atoms4_full, data_stream_custom_range, get_cosines
from montlake.geometry.geometry import get_geom, get_wlpca_tangent_sel, get_rm_tangent_sel
from montlake.gradients.estimate import get_grads_pullback
from montlake.statistics.normalization import normalize_L212
from montlake.optimization.gradientgrouplasso import get_sr_lambda_parallel
from montlake.optimization.utils import get_selected_function_ids,get_selected_functions_lm2
from montlake.utils.replicates import Replicate, get_supports_brute
from montlake.plotting.manifolds import plot_manifold_2d,plot_manifold_featurespace
from montlake.plotting.flasso import plot_reg_path_ax_lambdasearch_customcolors_norm, plot_watch_custom
from megaman.embedding import SpectralEmbedding
from montlake.statistics.supportrecovery import get_min_min, get_mu_full_ind, get_kappa_s,get_gamma_max

import pandas as pd
import dill as pickle
import os
import sys
import numpy as np
import itertools
from itertools import permutations,combinations
from sklearn.decomposition import TruncatedSVD
import pathos
from pathos.multiprocessing import ProcessingPool as Pool

In [ ]:
# export
import numpy as np
import random
import argparse
import json
import sys
import scipy
from ..vendor.tfcompat.hparam import HParams
from montlake.plotting.main import run_exp as run_exp_mflasso
from montlake.tslasso.main import run_exp as run_exp_tslasso
import os

def parse_args(args):
    parser= argparse.ArgumentParser()
    parser.add_argument("--config", help="Path to JSON config (can override with cmd line args)")
    parser.add_argument("--raw_data", help="Path to raw data")
    parser.add_argument("--outdir", help="Path to save outputs")
    args = parser.parse_args(args)
    return args

def subset_data(hparams):
    
    nsub = hparams.n
    file_type = hparams.raw_data[-3:]
    if file_type == "mat":
        data = scipy.io.loadmat(hparams.raw_data)
        n = data['R'].shape[0]
        nsub = np.min((nsub, n))
        randomindices = np.random.choice(range(n),nsub, replace = False)
        positions = data['R'][randomindices,:,:]        
    if file_type == "npz": 
        data = np.load(hparams.raw_data)
        n = data['R'].shape[0]
        nsub = np.min((nsub, n))
        randomindices = np.random.choice(range(n),nsub, replace = False)
        positions = data['R'][randomindices,:,:]  
    if file_type == 'npy':
        data = np.load(hparams.raw_data)
        n = data.shape[0]
        nsub = np.min((nsub, n))
        randomindices = np.random.choice(range(n),nsub, replace = False)
        positions = data[randomindices,:,:]  
    print(nsub, ' points avaiable')
    return(positions, randomindices)

In [ ]:
# export
import pdb

try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False
    
if __name__ == "__main__" and not IN_NOTEBOOK:
    
    np.random.seed(1234)
    random.seed(1234)
    args = parse_args(sys.argv[1:])
    print(args)
    config = {}
    if args.config:
        with open(args.config) as f:
            config.update(json.load(f))
            
    config.update(vars(args))
    hparams = HParams(**config)

    if not os.path.exists(hparams.outdir):
        os.makedirs(hparams.outdir)
        
#     with open(hparams.results,'rb') as inp:
#         results = pickle.load(inp, pickle.HIGHEST_PROTOCOL)
     
#     colors = np.asarray([])
#     for c in range(len(hparams.colors)):
#         colors.append(np.repeat(hparams.colors[c], hparams.color_count[c]))
    
    positions = np.load(hparams.position_file)
    
    plot_experiment(result_file = hparams.result_file, 
                    positions, 
                    d = hparams.d,
                    name = hparams.name, 
                    ncord = hparams.ncord,
                    embedding = hparams.embedding,
                    ground_truth = np.asarray(hparams.ground_truth, dtype = int), 
                    ground_truth_colors = hparams.ground_truth_colors,
                    outdir = outdir)
        